In [19]:
import requests
import pandas as pd
import numpy as np
import gzip
from io import BytesIO


In [20]:
df = pd.read_excel('eruptedFilamentsList.xlsx', sheet_name='Sheet1')  
SOLstr = df.iloc[1::2, 0].tolist()
# 提取时间部分并转换为 datetime 类型
time_ls = [pd.to_datetime(s[3:22], format='%Y-%m-%dT%H:%M:%S') for s in SOLstr]
print(time_ls)

[Timestamp('2010-05-26 06:18:58'), Timestamp('2010-05-31 15:05:42'), Timestamp('2010-06-01 16:15:39'), Timestamp('2010-06-03 00:00:02'), Timestamp('2010-06-10 19:47:53'), Timestamp('2010-06-16 13:45:00'), Timestamp('2010-06-22 08:36:44'), Timestamp('2010-06-22 20:37:58'), Timestamp('2010-06-22 20:37:58'), Timestamp('2010-06-23 13:48:25'), Timestamp('2010-06-27 01:06:44'), Timestamp('2010-06-27 01:06:44'), Timestamp('2010-06-29 14:31:39'), Timestamp('2010-07-03 03:55:45'), Timestamp('2010-07-03 07:33:04'), Timestamp('2010-07-06 00:22:40'), Timestamp('2010-07-06 08:18:59'), Timestamp('2010-07-08 05:55:17'), Timestamp('2010-07-17 18:17:42'), Timestamp('2010-07-19 01:40:23'), Timestamp('2010-07-20 07:04:07'), Timestamp('2010-07-21 00:00:29'), Timestamp('2010-07-30 06:52:08'), Timestamp('2010-08-01 00:00:31'), Timestamp('2010-08-01 00:00:31'), Timestamp('2010-08-01 10:17:50'), Timestamp('2010-08-01 15:39:39'), Timestamp('2010-08-23 11:46:40'), Timestamp('2010-08-26 12:00:00'), Timestamp('20

In [52]:
from datetime import timedelta
date_ls = [t.strftime('%Y%m%d') for t in time_ls]
date_ls = list(set(date_ls))
date_extend = []
for d in date_ls:
    date = pd.to_datetime(d, format='%Y%m%d')
    pre = date - timedelta(days=1)
    nxt = date + timedelta(days=1)
    date_extend.append(pre)
    date_extend.append(date)
    date_extend.append(nxt)
date_extend = list(set(date_extend))
date_extend.sort()
date_ls_extend = [d.strftime('%Y%m%d') for d in date_extend]

In [53]:
from bs4 import BeautifulSoup

linkls = []
cnt = 0
for date in date_ls_extend:
    # cnt += 1
    # if cnt > 10:
    #     break
    year = date[:4]
    url = f"http://cesar.kso.ac.at/halpha3a/{year}/{date}/processed/"
    # 使用requests库获取网页内容
    response = requests.get(url)

    # 确保请求成功
    #response.raise_for_status()

    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 提取所有的<a>标签
    links = soup.find_all('a')

    # 打印每个链接的href属性
    for link in links[5:]:
        lkdate = link.get('href')[14:22]+link.get('href')[23:29]
        linkls.append(lkdate)

with open('KSOlink.txt', 'w') as f:
    for item in linkls:
        f.write("%s\n" % item)

In [54]:
# 读入列表
lkdate = []

with open('KSOlink.txt', 'r') as f:
    for line in f:
        line = line.rstrip('\n')
        lktime = pd.to_datetime(line, format='%Y%m%d%H%M%S')
        lkdate.append(lktime)


In [55]:
def find_matches(time_ls, lkdate, delta):
    result = {}
    i = 0
    j = 0
    len_time_ls = len(time_ls)
    len_lkdate = len(lkdate)
    while i < len_time_ls:
        matches = []
        while j < len_lkdate and lkdate[j] < time_ls[i] - delta:
            j += 1
        while j < len_lkdate and lkdate[j] <= time_ls[i] + delta:
            matches.append(lkdate[j])
            j += 1
        result[time_ls[i]] = matches
        i += 1
    return result


In [60]:
# 不同间隔的数据缺失率
delta_ls = [timedelta(hours=n) for n in range(1, 25)]
for delta in delta_ls:
    result = find_matches(time_ls, lkdate, delta)
    zeros = 0
    zero_ls = []
    for item in result:
        res_ls = result[item]
        num = len(res_ls)
        if num == 0:
            zeros += 1
            zero_ls.append(item)
    print(f'{zeros/len(time_ls):.3f}', delta)
    


0.727 1:00:00
0.664 2:00:00
0.606 3:00:00
0.541 4:00:00
0.489 5:00:00
0.431 6:00:00
0.370 7:00:00
0.315 8:00:00
0.284 9:00:00
0.268 10:00:00
0.264 11:00:00
0.249 12:00:00
0.247 13:00:00
0.249 14:00:00
0.245 15:00:00
0.240 16:00:00
0.243 17:00:00
0.236 18:00:00
0.235 19:00:00
0.228 20:00:00
0.214 21:00:00
0.219 22:00:00
0.214 23:00:00
0.214 1 day, 0:00:00


In [70]:
import os
def download(time, srctime):
    timestr = time.strftime('%Y%m%d%H%M%S')
    srcstr = srctime.strftime('%Y%m%d%H%M%S')
    #格式如 http://cesar.kso.ac.at/halpha3a/2017/20170106/processed/kanz_halph_fi_20170106_080600.fts.gz
    ftsfile = f'kanz_halph_fi_{timestr[:8]}_{timestr[-6:]}.fts'

    url = f"http://cesar.kso.ac.at/halpha3a/{timestr[:4]}/{timestr[:8]}/processed/{ftsfile}.gz"

    path = f'./{srcstr}/'

    filepath = f'{path}{ftsfile}'

    os.makedirs(path, exist_ok=True)

    response = requests.get(url)
    
    with gzip.GzipFile(fileobj=BytesIO(response.content)) as gz:
        with open(filepath, 'wb') as out_f:
            out_f.write(gz.read())

def donwload_batch(list, srctime):
    for time in list:
        download(time, srctime)

In [71]:

delta = timedelta(hours=12)
result = find_matches(time_ls, lkdate, delta)
for item in result:
    res_ls = result[item]
    num = len(res_ls)
    if num == 0:
        continue
    if num <= 10:
        donwload_batch(res_ls, item)
    else:
        near_ls = sorted(res_ls, key=lambda x: abs(x - item))
        donwload_batch(near_ls[:10], item)
        


In [72]:
ha_ls = []
for item in result:
    if len(result[item]) > 0:
        ha_ls.append(item)

ha_ls.sort()



[Timestamp('2010-05-26 06:18:58'), Timestamp('2010-05-31 15:05:42'), Timestamp('2010-06-01 16:15:39'), Timestamp('2010-06-03 00:00:02'), Timestamp('2010-06-10 19:47:53'), Timestamp('2010-06-22 08:36:44'), Timestamp('2010-06-23 13:48:25'), Timestamp('2010-06-29 14:31:39'), Timestamp('2010-07-03 03:55:45'), Timestamp('2010-07-06 00:22:40'), Timestamp('2010-07-08 05:55:17'), Timestamp('2010-07-17 18:17:42'), Timestamp('2010-07-19 01:40:23'), Timestamp('2010-07-20 07:04:07'), Timestamp('2010-07-21 00:00:29'), Timestamp('2010-08-01 10:17:50'), Timestamp('2010-08-23 11:46:40'), Timestamp('2010-08-26 12:00:00'), Timestamp('2010-09-04 12:46:18'), Timestamp('2010-09-07 00:00:00'), Timestamp('2010-09-10 23:00:00'), Timestamp('2010-09-12 00:00:00'), Timestamp('2010-09-28 00:00:00'), Timestamp('2010-10-10 00:00:12'), Timestamp('2010-11-12 23:50:54'), Timestamp('2010-11-18 13:22:54'), Timestamp('2010-12-06 00:00:01'), Timestamp('2010-12-12 00:00:09'), Timestamp('2010-12-14 12:35:52'), Timestamp('20